# Create Master Bias and Subtract from the flat field and science images
This code will read in all of the raw data frames from Mookodi (MKD\*.fits) in the current direcdtory, create a master bias and then subtract this master bias from all of the flat field and science images in the same direcctory.

In [ ]:

import os
from pathlib import Path
import glob
import numpy as np
from astropy.io import fits


my_cwd = Path(os.getcwd())
fits_files = list(my_cwd.glob('MKD*.fits'))

bias_keyword = "BIAS" 

bias_files = []
files_for_bias_subtraction = []

# Make lists of the names of the bias images, and the flat field/science images

for file in fits_files:
    hdul = fits.open(file)  # open a FITS file
    hdr = hdul[0].header  #extract the header information 

    if hdr['IMAGETYP'] == bias_keyword and 'OBJECT'  in hdr :
       bias_files.append(file)
    else :
      files_for_bias_subtraction.append(file)


if len(bias_files) == 0:
       print("     !!! ERROR: no BIAS files found" )
       exit()



## Create the Master bias by taking the median of the individual bias frames
The mean and standard deviation of each individual bias frame are output to the screen.  Check that these numbers are consistent with each other. If there is a bad bias frame, delete its MKD\*.fits file and re-run the code. The new master bias frame will over-write the old master bias frame each time the cell below is run. The master bias frame has some keywords appended to its header to indicate it is a master bias, which was created by median combining a number of individual bias frames

In [ ]:
bias_concat = []

for i in bias_files:
    bias_hdulist = fits.open(i, ignore_missing_end=True)
    bias_hdu = bias_hdulist[0]
    bias_data = bias_hdu.data.astype(float)
    bias_concat.append(bias_data)

    bias_mean = np.mean(bias_data)
    bias_std = np.std(bias_data)
    file_name = os.path.basename(i)
    print(file_name, " mean: ", bias_mean, "stand. dev: ",bias_std)


# take median of all biases to make master bias
master_bias = np.median(bias_concat, axis=0)

#write out the bias frame, adding keywords 
new_hdu = bias_hdu
new_hdu.header['MASTERB'] = ("TRUE", "Master Bias")
new_hdu.header['NCOMBINE'] = (str(len(bias_files)), "Median Combine")
new_hdu.data = np.array((master_bias))
new_hdu.writeto("master_bias.fits", overwrite=True)
bias_hdulist.close()

print()
print("Created master_bias.fits and will now subtract it from the other frames")



## Subtract the master bias from all other fits frames in this directory
The bias subtracted frames will have filenames which start with a 'b'.  Their header will be updated to indicate that the frames have been bias subtracted.  

In [ ]:
#

print("New files created:") 

for i in files_for_bias_subtraction : 
    file_name = os.path.basename(i)
    outfile = "b" + file_name
    print(outfile) 
    object_hdulist = fits.open(i, ignore_missing_end=True)
    object_hdu = object_hdulist[0]
    object_data = object_hdu.data.astype(float) 
    new_hdu = object_hdu
    new_hdu.header['BIAS_SUB'] = ("TRUE", "Bias Subtracted")
    new_hdu.data = object_data - master_bias
    new_hdu.writeto(outfile, overwrite=True)
   
